In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import re
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
import io
import urllib.request
import requests


In [ ]:
def australia_trans_text(texts):
    

In [ ]:
def pdf_to_text(pdf_file):
    text_memory_file = io.StringIO()

    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, text_memory_file, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    for page in PDFPage.get_pages(pdf_file):
        interpreter.process_page(page)
    text = text_memory_file.getvalue()
    text_memory_file.close()
   
    return text

In [ ]:
def scrap_australia(cur_title):
    options = webdriver.ChromeOptions()
    options.add_argument('window-size=1920x1080')
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)
    
    driver.get('https://www.abf.gov.au/help-and-support/notices/australian-customs-notices')
    time.sleep(2)
    
    dates = []
    titles = []
    links = []
    last_page = len(temp)

    for i in range(2,last_page):
        soup = BeautifulSoup(driver.page_source, 'lxml')

        date_rows = soup.select('div.col-sm-3 span')
        title_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-header')})
        link_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-content')})

        for row in date_rows:
            date = row.text
            dates.append(date)

        for row in title_rows:
            tds = row.find_all('td')
            title = tds[1].get_text().replace('Press Enter to show more details.', '')  
            titles.append(title)

        for row in link_rows:
            link = 'https://www.abf.gov.au' + row.a['href']
            links.append(link)

        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(2)

        elem = driver.find_element(By.LINK_TEXT,str(i))
        elem.click()
        time.sleep(1)
        
    texts = []
    hscodes = []
    start = 0
    finish = len(links)

    for i in range(start, finish): 
        split_texts = []

        url = links[i]
        response = requests.get(url)
        my_raw_data = response.content

        # online pdf to text by requests
        response = requests.get(url)
        pdf_memory_file = io.BytesIO()
        pdf_memory_file.write(response.content)

        pdf_text = pdf_to_text(pdf_memory_file)
        pdf_text = pdf_text.lower()

        # HS CODE 가져오기
        hscode = re.findall('\d{4}\.\d{2}', pdf_text)
        hscodes.append(list(set(hscode)))

        #[signed] 찾기 위해 단어 단위로 쪼개기
        split_words = pdf_text.split()

        sentence = ""
        for word in range(len(split_words)):

            #[signed]가 있는 경우
            if split_words[word] == "[signed]":
                texts.append(sentence)
                break

            #assistant secretary가 있는 경우
            elif split_words[word] == "assistant" and len(split_words) > word+1:
                if split_words[word+1] == "secretary":
                    texts.append(sentence)
                    break
                else:
                    sentence = sentence + " " + split_words[word]

            #customs and trade policy branch가 있는 경우
            elif split_words[word]== "customs" and len(split_words) > word+4:
                if split_words[word+1]=="and" and split_words[word+2]=="trade" and split_words[word+3]=="policy" and split_words[word+4]=="branch":
                    texts.append(sentence)
                    break
                else:
                    sentence = sentence + " " + split_words[word]

            elif len(split_words) == word+1:
                sentence = sentence + " " + split_words[word]
                texts.append(sentence)

            else:
                sentence = sentence + " " + split_words[word]

        if len(texts)<i+1:
            texts.append("NONE")
            print(i+1, "번 문서 종료 / 총 길이는 : ", len(texts[i]))
        else:
            print(i+1, "번 문서 종료 / 총 길이는 : ", len(texts[i]))
    
    
    #
    title_trans = []
    for i in df["title"]:
        trans = get_translate(i)
        title_trans.append(trans)

In [ ]:
# title 한국어 번역 함수
def get_translate(text):
    client_id = "" # <-- client_id 기입
    client_secret = "" # <-- client_secret 기입

    data = {'text' : text,
            'source' : 'en',
            'target': 'ko'}

    url = "https://openapi.naver.com/v1/papago/n2mt"

    header = {"X-Naver-Client-Id":client_id,
              "X-Naver-Client-Secret":client_secret}

    response = requests.post(url, headers=header, data=data)
    rescode = response.status_code

    if(rescode==200):
        send_data = response.json()
        trans_data = (send_data['message']['result']['translatedText'])
        return trans_data
    else:
        print("Error Code:" , rescode)

In [ ]:
# hscode 데이터프레임 생성 함수
def make_df(hscodes):

    df_final = pd.DataFrame()

    hs_code = []

    for i in range(len(hscodes)):
        H = []
        h_result = ''
        df_check = pd.DataFrame()
        df_check['hscode'] = pd.Series(hscodes[i])

        H = df_check['hscode'].tolist()

        for h in H:
            h_result = h_result + "/" + h

        hs_code.append(h_result[1:])

        del df_check

    df_final['hscode'] = hs_code

    return df_final